<a href="https://colab.research.google.com/github/HarithaThiaguKumar/2019-Fall-Assignment-Look-and-feel/blob/master/covidtwitteranalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!ls '/content/drive/My Drive/Colab Notebooks/twitter.properties.txt'

'/content/drive/My Drive/Colab Notebooks/twitter.properties.txt'


In [3]:

!pip install ConfigParser

In [4]:
import configparser
config = configparser.RawConfigParser()
config.read('/content/drive/My Drive/Colab Notebooks/twitter.properties.txt')

print(config.sections());

['twitter']


In [5]:
import os
import tweepy as tw

In [6]:

accesstoken=config.get('twitter', 'accesstoken')
accesstokensecret=config.get('twitter', 'accesstokensecret')
apikey=config.get('twitter', 'apikey')
apisecretkey=config.get('twitter', 'apisecretkey')

In [7]:

auth = tw.OAuthHandler(apikey, apisecretkey)
auth.set_access_token(accesstoken, accesstokensecret)
api = tw.API(auth, wait_on_rate_limit=True)

In [8]:
search_words = "#coronavirus"
date_since = "2020-03-10"

In [9]:
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(1000)

In [10]:
tweets

In [11]:

tweet_details = [[tweet.geo, tweet.text,tweet.user.screen_name, tweet.user.location] for tweet in tweets]
#tweet_details

In [12]:
import pandas as pd

In [13]:
tweet_df = pd.DataFrame(data=tweet_details, columns=['geo','text','user', "location"])

In [14]:
pd.set_option('max_colwidth', 800)

tweet_df.head(20)

,geo,text,user,location
0,None,ALERT: Hillary Clinton: 'I Would Have Done a Better Job' than Trump\non Coronavirus - Global Pandemic News |… https://t.co/kgwgs4ycHJ,GlobalPandemics,WORLDWIDE
1,None,"RT @Mojahedineng: More than 64,800 people have died of the novel #coronavirus in 342 cities checkered across all of #Iran's 31 provinces, a…",Mohamma28307078,
2,None,"RT @AngelaRayner: Our cultural heritage has been hit hard with #coronavirus Theatres closing, live music decimated&amp;musicians struggling, ou…",danoneillwrites,Dublin
3,None,RT @RedPillMaC: The #WorldHealthOrganization made a complete U turn and said that #coronavirus patients doesn't need to be isolated or quar…,lifegarcia,
4,None,"RT @JohnNosta: BREAKING! Treatment with #Hydroxychloroquine , Azithromycin, and Combination in Patients Hospitalized with #COVID19 shows si…",Mobius22_,
5,None,"RT @daniellevitt22: #Lockdowns cannot logically work, as you can’t put the virus into a box and wait for it to disappear, it will just wait…",MineiroGiant,
6,None,RT @HLAurora63: 😳How many deaths is this man responsible for?\n**We’ll never know.\n\nHow many times has he changed his story?\n**Too many to c…,lori_tech,United States
7,None,RT @JohnBWellsCTM: BREAKING: \n\nThe #WorldHealthOrganization made a complete U turn and said that #coronavirus patients doesn't need to be i…,LizzieVasquez66,"California, USA"
8,None,RT @JohnBWellsCTM: BREAKING: \n\nThe #WorldHealthOrganization made a complete U turn and said that #coronavirus patients doesn't need to be i…,bryanlion8,"Colorado, USA"
9,None,RT @JohnBWellsCTM: BREAKING: \n\nThe #WorldHealthOrganization made a complete U turn and said that #coronavirus patients doesn't need to be i…,RainyThursday1,


In [15]:
tweet_df.user.value_counts()

ClippingBoss       9
Mohamma28307078    5
authorjdokerson    4
ZendeBaadAzadi     4
sophianictract     4
                  ..
54Ange             1
PonyboyAnon        1
clairemu1          1
LaShawnWms         1
amberwave          1
Name: user, Length: 925, dtype: int64

In [16]:
tweet_df.location.value_counts()

                           362
United States               27
California, USA             13
USA                         12
Texas, USA                  10
                          ... 
PA.                          1
Arizona                      1
Missouri, USA                1
Fresno & LA, California      1
Texas, USA                   1
Name: location, Length: 447, dtype: int64

In [17]:

import re
def clean_tweets(text):
    text = re.sub("RT @[\w]*:","",text)
    text = re.sub("@[\w]*","",text)
    text = re.sub("https?://[A-Za-z0-9./]*","",text)
    text = re.sub("\n","",text)
    return text

In [18]:

tweet_df['text']=tweet_df['text'].apply(lambda x: clean_tweets(x))

In [19]:
tweet_df.head(20)

,geo,text,user,location
0,None,ALERT: Hillary Clinton: 'I Would Have Done a Better Job' than Trumpon Coronavirus - Global Pandemic News |…,GlobalPandemics,WORLDWIDE
1,None,"More than 64,800 people have died of the novel #coronavirus in 342 cities checkered across all of #Iran's 31 provinces, a…",Mohamma28307078,
2,None,"Our cultural heritage has been hit hard with #coronavirus Theatres closing, live music decimated&amp;musicians struggling, ou…",danoneillwrites,Dublin
3,None,The #WorldHealthOrganization made a complete U turn and said that #coronavirus patients doesn't need to be isolated or quar…,lifegarcia,
4,None,"BREAKING! Treatment with #Hydroxychloroquine , Azithromycin, and Combination in Patients Hospitalized with #COVID19 shows si…",Mobius22_,
5,None,"#Lockdowns cannot logically work, as you can’t put the virus into a box and wait for it to disappear, it will just wait…",MineiroGiant,
6,None,😳How many deaths is this man responsible for?**We’ll never know.How many times has he changed his story?**Too many to c…,lori_tech,United States
7,None,BREAKING: The #WorldHealthOrganization made a complete U turn and said that #coronavirus patients doesn't need to be i…,LizzieVasquez66,"California, USA"
8,None,BREAKING: The #WorldHealthOrganization made a complete U turn and said that #coronavirus patients doesn't need to be i…,bryanlion8,"Colorado, USA"
9,None,BREAKING: The #WorldHealthOrganization made a complete U turn and said that #coronavirus patients doesn't need to be i…,RainyThursday1,


In [20]:
tweet_df.to_csv('tweets.csv')

In [21]:
!ls

drive  sample_data  tweets.csv


In [22]:

import spacy 
  
nlp = spacy.load('en_core_web_sm')

In [23]:
tweet_df['text'].apply(lambda x: [print("\tText : {}, Entity : {}".format(ent.text, ent.label_)) if (not ent.text.startswith('#')) else ""  for ent in nlp(x).ents])

	Text : Hillary Clinton, Entity : PERSON
	Text : Trumpon Coronavirus, Entity : ORG
	Text : More than 64,800, Entity : CARDINAL
	Text : 342, Entity : CARDINAL
	Text : Iran, Entity : GPE
	Text : 31, Entity : CARDINAL
	Text : decimated&amp;musicians, Entity : NORP
	Text : WorldHealthOrganization, Entity : MONEY
	Text : coronavirus, Entity : PERSON
	Text : Hydroxychloroquine, Entity : PERSON
	Text : Azithromycin, Entity : GPE
	Text : Lockdowns, Entity : PERSON
	Text : WorldHealthOrganization, Entity : MONEY
	Text : coronavirus, Entity : PERSON
	Text : WorldHealthOrganization, Entity : MONEY
	Text : coronavirus, Entity : PERSON
	Text : WorldHealthOrganization, Entity : MONEY
	Text : coronavirus, Entity : PERSON
	Text : The Secret Service, Entity : ORG
	Text : a day, Entity : DATE
	Text : two, Entity : CARDINAL
	Text : N95, Entity : ORG
	Text : WorldHealthOrganization, Entity : MONEY
	Text : coronavirus, Entity : PERSON
	Text : WorldHealthOrganization, Entity : MONEY
	Text : coronavirus, Ent

0                  [None, None]
1      [None, None, None, None]
2                        [None]
3                  [None, None]
4                  [None, None]
                 ...           
995                          []
996          [None, None, None]
997                          []
998                [None, None]
999                          []
Name: text, Length: 1000, dtype: object

In [24]:

tweet_df['entities']=tweet_df['text'].apply(lambda x: [(ent.text, ent.label_) if (not ent.text.startswith('#')) else "" for ent in nlp(x).ents])

In [25]:
tweet_df.head(20)

,geo,text,user,location,entities
0,None,ALERT: Hillary Clinton: 'I Would Have Done a Better Job' than Trumpon Coronavirus - Global Pandemic News |…,GlobalPandemics,WORLDWIDE,"[(Hillary Clinton, PERSON), (Trumpon Coronavirus, ORG)]"
1,None,"More than 64,800 people have died of the novel #coronavirus in 342 cities checkered across all of #Iran's 31 provinces, a…",Mohamma28307078,,"[(More than 64,800, CARDINAL), (342, CARDINAL), (Iran, GPE), (31, CARDINAL)]"
2,None,"Our cultural heritage has been hit hard with #coronavirus Theatres closing, live music decimated&amp;musicians struggling, ou…",danoneillwrites,Dublin,"[(decimated&amp;musicians, NORP)]"
3,None,The #WorldHealthOrganization made a complete U turn and said that #coronavirus patients doesn't need to be isolated or quar…,lifegarcia,,"[(WorldHealthOrganization, MONEY), (coronavirus, PERSON)]"
4,None,"BREAKING! Treatment with #Hydroxychloroquine , Azithromycin, and Combination in Patients Hospitalized with #COVID19 shows si…",Mobius22_,,"[(Hydroxychloroquine, PERSON), (Azithromycin, GPE)]"
5,None,"#Lockdowns cannot logically work, as you can’t put the virus into a box and wait for it to disappear, it will just wait…",MineiroGiant,,"[(Lockdowns, PERSON)]"
6,None,😳How many deaths is this man responsible for?**We’ll never know.How many times has he changed his story?**Too many to c…,lori_tech,United States,[]
7,None,BREAKING: The #WorldHealthOrganization made a complete U turn and said that #coronavirus patients doesn't need to be i…,LizzieVasquez66,"California, USA","[(WorldHealthOrganization, MONEY), (coronavirus, PERSON)]"
8,None,BREAKING: The #WorldHealthOrganization made a complete U turn and said that #coronavirus patients doesn't need to be i…,bryanlion8,"Colorado, USA","[(WorldHealthOrganization, MONEY), (coronavirus, PERSON)]"
9,None,BREAKING: The #WorldHealthOrganization made a complete U turn and said that #coronavirus patients doesn't need to be i…,RainyThursday1,,"[(WorldHealthOrganization, MONEY), (coronavirus, PERSON)]"


In [26]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


True

In [27]:
sid = SentimentIntensityAnalyzer()

In [28]:
tweet_df['sentiment']=tweet_df['text'].apply(lambda x: sid.polarity_scores(x))

In [29]:
tweet_df.head(10)

,geo,text,user,location,entities,sentiment
0,None,ALERT: Hillary Clinton: 'I Would Have Done a Better Job' than Trumpon Coronavirus - Global Pandemic News |…,GlobalPandemics,WORLDWIDE,"[(Hillary Clinton, PERSON), (Trumpon Coronavirus, ORG)]","{'neg': 0.0, 'neu': 0.706, 'pos': 0.294, 'compound': 0.7034}"
1,None,"More than 64,800 people have died of the novel #coronavirus in 342 cities checkered across all of #Iran's 31 provinces, a…",Mohamma28307078,,"[(More than 64,800, CARDINAL), (342, CARDINAL), (Iran, GPE), (31, CARDINAL)]","{'neg': 0.145, 'neu': 0.763, 'pos': 0.092, 'compound': -0.3182}"
2,None,"Our cultural heritage has been hit hard with #coronavirus Theatres closing, live music decimated&amp;musicians struggling, ou…",danoneillwrites,Dublin,"[(decimated&amp;musicians, NORP)]","{'neg': 0.231, 'neu': 0.769, 'pos': 0.0, 'compound': -0.4939}"
3,None,The #WorldHealthOrganization made a complete U turn and said that #coronavirus patients doesn't need to be isolated or quar…,lifegarcia,,"[(WorldHealthOrganization, MONEY), (coronavirus, PERSON)]","{'neg': 0.126, 'neu': 0.874, 'pos': 0.0, 'compound': -0.3182}"
4,None,"BREAKING! Treatment with #Hydroxychloroquine , Azithromycin, and Combination in Patients Hospitalized with #COVID19 shows si…",Mobius22_,,"[(Hydroxychloroquine, PERSON), (Azithromycin, GPE)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
5,None,"#Lockdowns cannot logically work, as you can’t put the virus into a box and wait for it to disappear, it will just wait…",MineiroGiant,,"[(Lockdowns, PERSON)]","{'neg': 0.083, 'neu': 0.917, 'pos': 0.0, 'compound': -0.2263}"
6,None,😳How many deaths is this man responsible for?**We’ll never know.How many times has he changed his story?**Too many to c…,lori_tech,United States,[],"{'neg': 0.0, 'neu': 0.877, 'pos': 0.123, 'compound': 0.3939}"
7,None,BREAKING: The #WorldHealthOrganization made a complete U turn and said that #coronavirus patients doesn't need to be i…,LizzieVasquez66,"California, USA","[(WorldHealthOrganization, MONEY), (coronavirus, PERSON)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
8,None,BREAKING: The #WorldHealthOrganization made a complete U turn and said that #coronavirus patients doesn't need to be i…,bryanlion8,"Colorado, USA","[(WorldHealthOrganization, MONEY), (coronavirus, PERSON)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
9,None,BREAKING: The #WorldHealthOrganization made a complete U turn and said that #coronavirus patients doesn't need to be i…,RainyThursday1,,"[(WorldHealthOrganization, MONEY), (coronavirus, PERSON)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"


In [30]:

tweet_df.dtypes

geo          object
text         object
user         object
location     object
entities     object
sentiment    object
dtype: object

In [31]:
tweet_df.to_csv("tweets.csv")